# OptStab Plots of Unpacking

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from OptStab import beta
from scipy.interpolate import interp1d

%matplotlib inline
VERSION = 1

from os.path import expanduser
home = expanduser("~")
FIGURE_DIR = os.path.join(home,'Scratch','OptStabFigures')

PRESENTATION = False
MODE = 'b'
OutDir = os.path.join(os.path.pardir,os.path.pardir,'data','results')

# Scale the RGB values to the [0, 1] range, which is the format matplotlib accepts.    
def ColorScale(colors):
    for (i,(r, g, b)) in enumerate(colors):
        colors[i] = (r / 255., g / 255., b / 255.)
        
    return colors


if PRESENTATION:
    mycolors = ColorScale([(51,102,204), (153,0,0), (0,153,0), (153,0,0), (51,102,204)]  )
    mylinestyles = ['-','-','-','--','--']
else:
    mycolors = ColorScale([3*(0,),3*(0,),3*(0,),3*(170,),3*(170,) ])
    mylinestyles = ['-','--','-.','-','--']

    
def FormatFig(hndls, colors = mycolors, linestyles = mylinestyles):
    for (i,h) in enumerate(hndls):
        h.set_linestyle(linestyles[i])
        h.set_color(colors[i])
        
if MODE == 'b':
    xlab = r'UI generosity ($b$)'
else:
    xlab = r'Tax progressivity ($\tau$)'    

In [ ]:
with np.load(os.path.join(OutDir,'Unpack_v'+str(VERSION)+'_param_' + MODE +'_mode_fig.npz')) as X:
    ExpectedMarginalWelfare = X['ExpectedMarginalWelfare']
    Cov = X['Cov']
    EdwEdx = X['EdWdxEdxdparam']
    paramgrid = X['paramgrid'][:-1]
    

# check if the figure directory exists and create it if it doesn't
if not os.path.isdir(FIGURE_DIR):
    os.makedirs(FIGURE_DIR)
    
if MODE == 'b':
    parind = 0
else:
    parind = 1
        
# convert to marginal consumption equivalents        
ExpectedMarginalWelfare = (1-beta) * ExpectedMarginalWelfare
#--debug--
from OptStab import delta
# ExpectedMarginalWelfare[:,0] *= (1-beta*(1-delta))/(1-beta)
#--debug--
Cov = (1-beta) * Cov
EdwEdx = (1-beta) * EdwEdx

In [ ]:
if PRESENTATION:
    fig = plt.figure(figsize=(8, 6))
else:
    fig = plt.figure()
    
ax = plt.subplot(111)
# ax.spines["top"].set_visible(False)    
# ax.spines["bottom"].set_visible(False)    
# ax.spines["right"].set_visible(False)    
# ax.spines["left"].set_visible(False)

# Ensure that the axis ticks only show up on the bottom and left of the plot.    
# Ticks on the right and top of the plot are generally unnecessary chartjunk.    
ax.get_xaxis().tick_bottom()    
ax.get_yaxis().tick_left()  

# Make sure your axis ticks are large enough to be easily read.    
# You don't want your viewers squinting to read your plot.    
# plt.xticks(np.linspace(b_grid.min(), b_grid.max(), 8), [str(x)  for x in np.linspace(b_grid.min(), b_grid.max(), 8)], fontsize=14)    


hndls = plt.plot(paramgrid,np.hstack((ExpectedMarginalWelfare[:,[0,1]],
                                      ExpectedMarginalWelfare[:,2:].sum(axis=1)[np.newaxis].T - Cov[:,2:].sum(axis=1)[np.newaxis].T,
                                     Cov[:,2:].sum(axis=1)[np.newaxis].T)))


plt.xlabel(xlab,fontsize =14,family='serif')
plt.ylabel('Marginal welfare gain',fontsize =14,family='serif')

if PRESENTATION:
    colors = ColorScale([(51,102,204), (153,0,0), (0,153,0), (0,153,0)]  )
    linestyles = ['-','-','-','--']
    FormatFig(hndls, colors = colors, linestyles = linestyles)
else:
    FormatFig(hndls)
    
L = plt.legend(hndls,['Insurance', 'Incentives', 'Macro stab. E x E','Macro stab. Covariance'],fontsize = 14,bbox_to_anchor=(1.04,0.5), loc="center left")
plt.setp(L.texts, family='serif')
plt.ylim([-0.05,0.05])

    
fig.savefig(os.path.join(FIGURE_DIR,'UnpackFourTerms_'+MODE+'.png'),bbox_inches = 'tight')



In [ ]:
if PRESENTATION:
    fig = plt.figure(figsize=(8, 6))
else:
    fig = plt.figure()
    
ax = plt.subplot(111)
hndls = plt.plot(paramgrid,np.hstack((ExpectedMarginalWelfare[:,2:5],
                           ExpectedMarginalWelfare[:,5:].sum(axis=1)[np.newaxis].T)))

if PRESENTATION:
    colors = [(51,102,204), (153,0,0), (0,153,0), (0,0,0)]  
    for (i,(r, g, b)) in enumerate(colors):
        colors[i] = (r / 255., g / 255., b / 255.)
    linestyles = ['-','-','-','-']
    FormatFig(hndls,colors = colors, linestyles = linestyles)
else:
    FormatFig(hndls)

L = plt.legend(hndls,['Labor wedge','Price dispersion', 'Extensive margin', 'Idio. risk'],fontsize = 14,bbox_to_anchor=(1.04,0.5), loc="center left")
plt.setp(L.texts, family='serif')
plt.xlabel(xlab,fontsize =14,family='serif')
plt.ylabel('Marginal welfare gain',fontsize =14,family='serif')
fig.savefig(os.path.join(FIGURE_DIR,'UnpackMacro_'+MODE+'.png'),bbox_inches = 'tight')

In [ ]:
# Plot more detail for review
hndls = plt.plot(paramgrid,ExpectedMarginalWelfare[:,5:])
L = plt.legend(hndls,['Unemployment risk', 'Skill risk'],fontsize = 14)

In [ ]:
plt.plot(paramgrid,ExpectedMarginalWelfare.sum(axis=1))

In [ ]:
interp1d(ExpectedMarginalWelfare.sum(axis=1),paramgrid)(0.0)

In [ ]:
# Ratio of cov to incentives term
plt.plot(paramgrid,Cov[:,2:].sum(axis=1)[np.newaxis].T/ np.abs(ExpectedMarginalWelfare[:,[1]] ))
# plt.ylim([1.5,2])
# plt.xlim([0.82,0.828])